In [2]:
import pandas as pd

### Majors, 25- 50- 75- salaries

In [12]:
majors_df = pd.read_csv("raw/all-ages.csv")
majors_df.head()
majors_df = majors_df.rename(columns = {"Major" : "Name", "Median" : "50_Salary", "P25th" : "25_Salary", "P75th" : "75_Salary"})
majors_df = majors_df[["Name", "25_Salary", "50_Salary", "75_Salary"]]
majors_df['75_Salary'] = majors_df['75_Salary'].astype(int)
majors_df['Name'] = majors_df['Name'].str.title()
majors_df.to_csv("final/subjects.csv", index=False)

### countries

In [43]:
countries_df = pd.read_csv("raw/Quality of life in a country comparison.csv")
countries_df.head()
countries_df = countries_df.rename(columns = {"Country" : "Name", "Rights(20%)" : "RightsScore", "Safety(10%)" : "SafetyScore", "Climate(15%)" : "ClimateScore", "Costs(15%)" : "LivingCostScore"})
final_countries_df = countries_df[["Name", "LivingCostScore", "SafetyScore", "RightsScore", "ClimateScore"]]
final_countries_df.to_csv("final/countries.csv", index=False)

In [137]:
times = pd.read_csv("raw/TIMES_WorldUniversityRankings_2024.csv")
qs = pd.read_csv("raw/qs2024.csv")

In [138]:
times = times.rename(columns = {"name" : "Name", "scores_overall" : "OverallScore", "scores_teaching" : "TeachingScore", "scores_research": "ResearchScore", "stats_number_students" : "Population"})
qs = qs.rename(columns = {"Institution Name" : "Name", "Country" : "Location", "Employer Reputation Score" : "EmployerScore"})
times = times[["Name", "OverallScore", "TeachingScore", "ResearchScore", "Population"]]
qs = qs[["Name", "Location", "EmployerScore"]]
times = times.dropna()
qs = qs.dropna()

In [139]:
qs['Name'] = qs['Name'].str.replace(r'\s*\([^)]*\)\s*', '', regex=True)
qs['Name'] = qs['Name'].str.replace(r'^\s*The\s+', '', regex=True)
times['Name'] = times['Name'].str.replace(r'^\s*The\s+', '', regex=True)

In [171]:
result = pd.merge(times, qs, on='Name', how='right')

result2 = result
def replace(value):
    if pd.notna(value) and '–' in value:
        parts = value.split('–')
        average = round((float(parts[0]) + float(parts[1])) / 2, 1)
        return str(average)
    else:
        return value
    
result2['OverallScore'] = result2['OverallScore'].apply(replace).astype(float)

def convert(value):
    if pd.notna(value):
        return int((value.replace(',', '')))
    else:
        return value

result2['Population'] = result2['Population'].apply(convert)

result2.to_csv("final/universities.csv", index=False)

In [177]:
result2 = result2[["Name", "Location", "OverallScore", "TeachingScore", "ResearchScore", "EmployerScore", "Population"]]
result2.to_csv("final/universities.csv", index=False)

In [227]:
t2 = pd.read_csv("raw/TIMES_WorldUniversityRankings_2024.csv")
t2 = t2.rename(columns={"name" : "Name", "subjects_offered": "Offered"})
t2 = t2[["Name", "Offered"]]
t2 = t2.dropna()
t2['Name'] = t2['Name'].str.replace(r'^\s*The\s+', '', regex=True)

rows = []
for index, row in t2.iterrows():
    university = row['Name']
    majors = row['Offered'].split(',')
    for major in majors:
        rows.append({'Name': university, 'Subject': major})

newt2 = pd.DataFrame(rows)

In [228]:
newt2['Name'] = newt2['Name'].str.replace(r'\s*\([^)]*\)\s*', '', regex=True)
newt2['Subject'] = newt2['Subject'].str.replace('&', 'And')
newt2

,Name,Subject
0,University of Oxford,Geography
1,University of Oxford,Chemistry
2,University of Oxford,Chemical Engineering
3,University of Oxford,Biological Sciences
4,University of Oxford,General Engineering
...,...,...
58138,Ziane Achour University of Djelfa,Geology
58139,Ziane Achour University of Djelfa,Environmental
58140,Ziane Achour University of Djelfa,Earth And Marine Sciences
58141,Ziauddin University,Medicine And Dentistry


In [229]:
out = pd.merge(result2, newt2, on='Name', how='inner')
out = out[["Name", "Subject"]]
majors_df = majors_df.rename(columns = {"Name": "Subject"})
out2 = pd.merge(majors_df, out, on='Subject', how='inner')
out2 = out2[["Name", "Subject"]]
out2 = out2.rename(columns = {"Name" : "University"})
out2.to_csv("final/offeredsubjects.csv", index=False)

,University,Subject
0,Massachusetts Institute of Technology,Architecture
1,University of Cambridge,Architecture
2,Harvard University,Architecture
3,National University of Singapore,Architecture
4,UCL,Architecture
...,...,...
6151,University of Ibadan,History
6152,University of Lagos,History
6153,University of Lampung,History
6154,University of Oradea,History


In [230]:
out2.to_csv("final/offeredsubjects.csv", index=False)

In [257]:
qs17 = pd.read_csv("raw/qs1722.csv")
qs24 = pd.read_csv("raw/qs2024.csv")

In [258]:
qs17 = qs17.rename(columns={"university" : "University", "year" : "Year", "rank_display" : "Ranking"})
qs17 = qs17[["University", "Year", "Ranking"]]
qs17 = qs17.dropna()
qs17['University'] = qs17['University'].str.replace(r'\s*\([^)]*\)\s*', '', regex=True)
qs17['University'] = qs17['University'].str.replace(r'^\s*The\s+', '', regex=True)
qs17

,University,Year,Ranking
0,Massachusetts Institute of Technology,2017,1
1,Stanford University,2017,2
2,Harvard University,2017,3
3,University of Cambridge,2017,4
4,California Institute of Technology,2017,5
...,...,...,...
6477,Université de Tunis,2022,1201
6478,Université de Tunis El Manar,2022,1201
6479,Yarmouk University,2022,1201
6480,Yildiz Technical University,2022,1201


In [259]:
qs24 = qs24.rename(columns={"Institution Name" : "University"})
qs24 = qs24[["University", "2024 RANK", "2023 RANK"]]
qs24 = qs24.dropna()
qs24['University'] = qs24['University'].str.replace(r'\s*\([^)]*\)\s*', '', regex=True)
qs24['University'] = qs24['University'].str.replace(r'^\s*The\s+', '', regex=True)
qs24['2023 RANK'] = qs24['2023 RANK'].str.replace('+', '')
qs24['2024 RANK'] = qs24['2024 RANK'].str.replace('+', '')
qs24['2023 RANK'] = qs24['2023 RANK'].str.split('-').str[-1].str.strip()
qs24['2024 RANK'] = qs24['2024 RANK'].str.split('-').str[-1].str.strip()

rows = []
for index, row in qs24.iterrows():
    university = row['University']
    r23 = row['2023 RANK']
    r24 = row['2024 RANK']
    rows.append({'University': university, 'Year': 2023, 'Ranking': r23})
    rows.append({'University': university, 'Year': 2024, 'Ranking': r24})

final24 = pd.DataFrame(rows)
final24

/var/folders/wn/gqqc68k90tq6p5x47wyh_npw0000gn/T/ipykernel_96417/3257094064.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  qs24['2023 RANK'] = qs24['2023 RANK'].str.replace('+', '')
/var/folders/wn/gqqc68k90tq6p5x47wyh_npw0000gn/T/ipykernel_96417/3257094064.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  qs24['2024 RANK'] = qs24['2024 RANK'].str.replace('+', '')


,University,Year,Ranking
0,Massachusetts Institute of Technology,2023,1
1,Massachusetts Institute of Technology,2024,1
2,University of Cambridge,2023,2
3,University of Cambridge,2024,2
4,University of Oxford,2023,4
...,...,...,...
2825,University of Oradea,2024,1401
2826,"University Politehnica of Timisoara, UPT",2023,1400
2827,"University Politehnica of Timisoara, UPT",2024,1401
2828,Western Washington University,2023,1400


In [270]:
combined = pd.concat([qs17, final24])
combined
result3 = result2.rename(columns={"Name": "University"})
finalfinal = pd.merge(result3, combined, on="University", how="inner")
finalfinal = finalfinal[["University", "Year", "Ranking"]]
finalfinal.to_csv("final/rankings.csv")

In [280]:
check = pd.read_csv("final/rankings.csv")
check.dtypes

University    object
Year           int64
Ranking        int64
dtype: object

In [279]:
check = check[["University", "Year", "Ranking"]]
check.to_csv("final/rankings.csv", index=False)